<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/dfs_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class BFS {
    // The BFS class has a method named solve that takes three parameters: start, target, and grid.
    solve(start, target, grid) {
        // Initialize an array to keep track of the animation order.
        const animationOrder = [];
        // Initialize a queue with the start node and its path.
        const queue = [[start, [start]]];

        // Start a while loop that continues until the queue is not empty.
        while (queue.length) {
            // Dequeue the front element of the queue, which is an array containing the current node and its path.
            const [current, path] = queue.pop();

            // Check if the current node is the target node.
            if (current.x === target.x && current.y === target.y) {
                // If yes, return the animation order and the path.
                return [animationOrder, path];
            }

            // Get the neighbors of the current node using the neighbours method, passing the grid.
            current.neighbours(grid)
                // Map over the neighbors.
                .map((cell, j) => {
                    // Check if the neighbor has not been visited.
                    if (!cell.visited) {
                        // Mark the neighbor as visited.
                        cell.visited = true;
                        // Push the current node to the animation order array.
                        animationOrder.push(current);
                        // Enqueue the neighbor and its path to the front of the queue.
                        queue.unshift([cell, [...path, cell]]);
                    }
                });
        }
    }
}

// This class is missing a definition for the `neighbours` method, which is used in the solve method.






// Define a class named Cell.
class Cell {

    /**
     * Creates an instance of Cell.
     * @param {*} x
     * @param {*} y
     * @param {number} [d=1]
     * @memberof Cell
     */
    // Constructor function for creating instances of the Cell class.
    constructor(x, y, d = 1) {
        // Initialize the visited property to false.
        this.visited = false;
        // Set the x and y coordinates of the cell.
        this.x = x;
        this.y = y;
        // Set the radius of the cell based on the provided diameter (default is 1).
        this.r = d * 0.5;
        // Set the cost property of the cell to a random integer between 1 and 3 (inclusive).
        this.cost = getRandomInt(1, 3);
    }

    /**
     * Get neighbour cells
     *
     * @param {Grid} grid
     * @param {boolean} [filterWalls=true]
     * @return {[Cell]}
     * @memberof Cell
     */
    // Method for getting neighboring cells of the current cell.
    neighbours(grid, filterWalls = true) {
        // Define possible actions for neighboring cells in horizontal and vertical directions.
        const actions = [
            [1, 0],
            [-1, 0],
            [0, 1],
            [0, -1],
        ];
        // Define possible actions for neighboring cells in diagonal directions.
        const diagonals = [
            [1, 1],
            [-1, -1],
            [1, -1],
            [-1, 1]
        ];

        // Check if the grid allows diagonal movements.
        if (grid.hasDiagonals) {
            // If yes, combine both horizontal/vertical and diagonal actions.
            this.neighboursCells = [...actions, ...diagonals];
        } else {
            // If no diagonal movements, use only horizontal/vertical actions.
            this.neighboursCells = actions;
        }

        // Map over the possible neighboring coordinates, filter out invalid coordinates, and get corresponding cell objects.
        this.neighboursCells = this.neighboursCells
            .map(c => [this.x + c[0], this.y + c[1]])
            .filter(coord => grid.isValid(...coord))
            .filter(coord => filterWalls ? !grid.isWall(...coord) : !filterWalls)
            .map(coord => grid.getCell(...coord));

        // Return the array of neighboring cell objects.
        return this.neighboursCells;
    }
}

// Note: The code assumes there is a function `getRandomInt` defined elsewhere in your code, as it is used in the constructor.



// Define a class named Wall that extends the Cell class.
class Wall extends Cell {

    /**
     * Creates an instance of Wall.
     * @param {*} x
     * @param {*} y
     * @memberof Wall
     */
    // Constructor function for creating instances of the Wall class.
    constructor(x, y) {
        // Call the constructor of the base class (Cell) using the super keyword.
        super(x, y);
    }
}



// Define a class named Grid.
class Grid {

    /**
     * Creates an instance of Grid.
     * @param {Number} w
     * @param {Number} h
     * @memberof Grid
     */
    // Constructor function for creating instances of the Grid class.
    constructor(w, h) {
        // Initialize the cells property as a 2D array of Cell objects based on width (w) and height (h).
        this.cells = Array(h).fill().map((_, y) => {
            return Array(w).fill().map((_, x) => new Cell(x, y));
        });
    }

    /**
     *Create a maze from the grid
     *
     * @param {Number} w
     * @param {Number} h
     * @memberof Grid
     */
    // Method for creating a maze using a MazeGenerator instance.
    maze(w, h) {
        // Create a MazeGenerator instance with width (w) and height (h).
        const m = new MazeGenerator(w, h);
        // Generate the maze.
        m.make();
        // Set the cells property of the grid to the cells of the generated maze.
        this.cells = m.grid.cells;
    }

    /**
     *Get a cell from x and y coords
     *
     * @param {Number} x
     * @param {Number} y
     * @return {Cell | undefined}
     * @memberof Grid
     */
    // Method for getting a cell from coordinates (x, y).
    getCell(x, y) {
        // Check if the coordinates are valid, and return the corresponding cell or undefined.
        if (this.isValid(x, y))
            return this.cells[y][x];
    }

    /**
     *Check coordinates are valid
     *
     * @param {Number} x
     * @param {*Number} y
     * @return {boolean} is valid
     * @memberof Grid
     */
    // Method for checking if coordinates are valid within the grid.
    isValid(x, y) {
        // Return true if coordinates are within the grid bounds; otherwise, return false.
        return y >= 0 && y < this.cells.length && x >= 0 && x < this.cells[0].length;
    }

    /**
     *Check the given coordinates are a wall
     *
     * @param {Number} x
     * @param {Number} y
     * @return {boolean | undefined} true or false if it is valid, undefined if invalid
     * @memberof Grid
     */
    // Method for checking if the cell at given coordinates is a Wall.
    isWall(x, y) {
        // Check if the coordinates are valid and if the cell is an instance of the Wall class.
        if (this.isValid(x, y))
            return this.cells[y][x] instanceof Wall;
    }

    /**
     *randomise the grid
     *
     * @param {Number} [chance=.7]
     * @memberof Grid
     */
    // Method for randomizing the grid with a given chance.
    randomize(chance = 0.7) {
        // Iterate over each row and column, replacing cells with Walls based on the random chance.
        this.cells.forEach((row, y) => {
            row.forEach((_, x) => {
                const c = Math.random() > chance ? new Wall(x, y) : new Cell(x, y);
                this.cells[y][x] = c;
            });
        });
    }

    /**
     *Add some random obstacles and fixed distance obstacles
     *
     * @memberof Grid
     */
    // Method for adding a combination of random and fixed obstacles to the grid.
    combo() {
        // Randomize the grid with a higher chance of Walls.
        this.randomize(0.9);
        // Add obstacles with a size based on the user input or a default size of 8.
        this.obstacles(Number(document.getElementById('obstaclesize').value) ?? 8);
    }

    /**
     *Add some fixed distance obstacles
     *
     * @param {number} [size=8]
     * @memberof Grid
     */
    // Method for adding fixed distance obstacles to the grid.
    obstacles(size = 8) {
        // Calculate the size of obstacles based on the grid dimensions.
        size = Math.min(Math.floor(this.cells.length / 3), size);
        const splitX = Math.floor(this.cells[0].length / size) ?? 1;
        const splitY = Math.floor(this.cells.length / size) ?? 1;

        // Iterate over each cell, placing Walls at fixed distances.
        this.cells.forEach((row, y) => {
            row.forEach((_, x) => {
                if (x % splitX === 0) {
                    if (y % splitY === 0 || y % splitY === 1) {
                        this.cells[y][x] = new Wall(x, y);
                    }
                }
            });
        });
    }
}






// Function to get a random integer between min (inclusive) and max (exclusive).
function getRandomInt(min, max) {
    min = Math.ceil(min);
    max = Math.floor(max);
    return Math.floor(Math.random() * (max - min) + min); // The maximum is exclusive and the minimum is inclusive
}

// Function to generate a random RGB color string.
function getRandomColor() {
    return `rgb(${Math.floor(Math.random() * 256)}, ${Math.floor(Math.random() * 256)}, ${Math.floor(Math.random() * 256)})`;
}

// Function to get and store a random color associated with a key.
function getAndStoreColour() {
    const store = {};

    // Returns a function that, given a key, returns a stored color or generates and stores a new one.
    return (key) => {
        if (!store[key]) {
            store[key] = getRandomColor();
        }
        return store[key];
    };
}

// Initialize the getColour function using getAndStoreColour.
const getColour = getAndStoreColour();

// Chebyshev distance calculation function between two points.
function chebyshev(start, end) {
    return Math.max(Math.pow(start.x - end.x, 2), Math.pow(start.y - end.y, 2));
}

// Euclidean distance calculation function between two points.
function euclidian(start, end) {
    return Math.sqrt(Math.pow(start.x - end.x, 2) + Math.pow(start.y - end.y, 2));
}

// Manhattan distance calculation function between two points.
function manhattan(start, end) {
    return Math.abs((start.x - end.x) + (start.y - end.y));
}

// Function to get a dataset for charting based on specified axes and options.
function getDataset(axes) {
    // Retrieve performance data from localStorage.
    const perf = JSON.parse(localStorage.getItem('performance'));

    // Initialize an object to store datasets categorized by algorithm.
    const datasetsByAlgorithm = {};

    // Iterate over performance data to organize it by algorithm and convert axis values if specified.
    perf.forEach(item => {
        const algorithmKey = item.solver;
        let yAxis = item[axes.y];
        let xAxis = item[axes.x];
        let logBaseX = axes.logBaseX ?? 2;
        let logBaseY = axes.logBaseY ?? 2;

        // Create a dataset for the algorithm if not already present.
        if (!datasetsByAlgorithm[algorithmKey]) {
            datasetsByAlgorithm[algorithmKey] = {
                label: algorithmKey,
                data: [],
                pointBackgroundColor: getColour(algorithmKey),
            };
        }

        // Apply logarithmic transformation to axis values if specified.
        if (axes.log === 'y' || axes.log === 'both') {
            yAxis = Math.log(yAxis) / Math.log(logBaseY);
        }
        if (axes.log === 'x' || axes.log === 'both') {
            xAxis = Math.log(xAxis) / Math.log(logBaseX);
        }

        // Add the transformed data point to the dataset.
        datasetsByAlgorithm[algorithmKey].data.push({
            y: yAxis,
            x: xAxis,
        });
    });

    // Return the organized datasets by algorithm.
    return datasetsByAlgorithm;
}

